In [1]:
import torch
from torch import nn
import d2l.torch as d2l

In [2]:
def vgg_block(num_convs, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.LazyConv2d(out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

In [3]:
class VGG(d2l.Classifier):
    def __init__(self, arch, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        conv_blks = []
        for (num_convs, out_channels) in arch:
            conv_blks.append(vgg_block(num_convs, out_channels))
        self.net = nn.Sequential(
            *conv_blks, nn.Flatten(),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(4096), nn.ReLU(), nn.Dropout(0.5),
            nn.LazyLinear(num_classes))
        self.net.apply(d2l.init_cnn)

In [4]:
VGG(arch=((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))).layer_summary(
    (1, 1, 224, 224))

Sequential output shape:	 torch.Size([1, 64, 112, 112])
Sequential output shape:	 torch.Size([1, 128, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 28, 28])
Sequential output shape:	 torch.Size([1, 512, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
Flatten output shape:	 torch.Size([1, 25088])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 10])


In [5]:
model = VGG(arch=((1, 16), (1, 32), (2, 64), (2, 128), (2, 128)), lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(224, 224))

In [6]:
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data)

KeyboardInterrupt: 

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Create a VGG model instance
vgg_model = VGG(arch=((1, 16), (1, 32), (2, 64), (2, 128), (2, 128)))

# Initialize the model with a sample input
sample_input = torch.randn(1, 1, 224, 224)
_ = vgg_model(sample_input)

# Count and print the number of trainable parameters
num_params = count_parameters(vgg_model)
print(f"The VGG model has {num_params:,} trainable parameters.")

# You can also break it down by layer
print("\nParameter count by layer:")
for name, module in vgg_model.named_modules():
    if isinstance(module, (nn.Conv2d, nn.Linear)):
        print(f"{name}: {count_parameters(module):,}")


The VGG model has 43,093,322 trainable parameters.

Parameter count by layer:
net.0.0: 160
net.1.0: 4,640
net.2.0: 18,496
net.2.2: 36,928
net.3.0: 73,856
net.3.2: 147,584
net.4.0: 147,584
net.4.2: 147,584
net.6: 25,694,208
net.9: 16,781,312
net.12: 40,970
